## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

## Loading Dataset_1

In [ ]:
maternal_data = pd.read_csv("/content/Maternal Health Risk Data Set.csv")
maternal_data.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [ ]:
maternal_data.shape

(1014, 7)

In [ ]:
maternal_data['RiskLevel'].value_counts()

low risk     406
mid risk     336
high risk    272
Name: RiskLevel, dtype: int64

## Loading Dataset_2
Load maternal and fetal health datasets. Each will be modeled independently and later combined for final risk prediction.

In [ ]:
fetal_data = pd.read_csv("/content/fetal_health.csv")
fetal_data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [ ]:
fetal_data.shape

(2126, 22)

In [ ]:
fetal_data['fetal_health'].value_counts()

1.0    1655
2.0     295
3.0     176
Name: fetal_health, dtype: int64

## Dataset Resampling

The datasets are imbalanced, so I use upsampling (resample) to balance class distributions in both datasets. This prevents bias in training.

In [ ]:
low_risk = maternal_data[maternal_data['RiskLevel'] == 'low risk']
mid_risk = maternal_data[maternal_data['RiskLevel'] == 'mid risk']
high_risk = maternal_data[maternal_data['RiskLevel'] == 'high risk']
print(low_risk.shape)
print(mid_risk.shape)
print(high_risk.shape)

(406, 7)
(336, 7)
(272, 7)


In [ ]:
high_risk_upsampled = resample(high_risk, replace=True, n_samples=1655, random_state=42)
mid_risk_upsampled = resample(mid_risk, replace=True, n_samples=1655, random_state=42)
low_risk_upsampled = resample(low_risk, replace=True, n_samples=1655, random_state=42)


In [ ]:
maternal_data_upsampled= pd.concat([low_risk_upsampled, mid_risk_upsampled, high_risk_upsampled])

In [ ]:
maternal_data_upsampled['RiskLevel'].value_counts()

low risk     1655
mid risk     1655
high risk    1655
Name: RiskLevel, dtype: int64

In [ ]:
print(maternal_data_upsampled)

     Age  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate  RiskLevel
251   12          95           60   6.9      98.0         65   low risk
896   15         120           80   7.9      98.0         70   low risk
574   23          90           60   7.5      98.0         76   low risk
255   25         100           90   6.9      98.0         76   low risk
180   12          95           60   6.1     102.0         60   low risk
..   ...         ...          ...   ...       ...        ...        ...
486   38         135           60   7.9     101.0         86  high risk
456   23         140           90   6.8      98.0         70  high risk
532   49         140           90  15.0      98.0         90  high risk
102   48         140           90  15.0      98.0         90  high risk
440   23         140           90   6.8      98.0         70  high risk

[4965 rows x 7 columns]


## Data Shuffling_Maternal
Shuffle the resampled datasets to remove ordering bias before training.

In [ ]:
maternal_data = maternal_data_upsampled.sample(frac=1)
print(maternal_data)
maternal_data.shape

     Age  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate  RiskLevel
374   50         130          100  16.0      98.0         75  high risk
410   19         120           90   6.8      98.0         60   mid risk
84    21          90           65   6.9      98.0         76   mid risk
676   29          90           70   8.0     100.0         80  high risk
297   42         120           80   7.7      98.0         70   low risk
..   ...         ...          ...   ...       ...        ...        ...
618   23         120           90   7.5      98.0         70   low risk
67    19         120           80   7.0      98.0         70   mid risk
16    50         140           90  15.0      98.0         90  high risk
505   17          70           50   7.9      98.0         70   low risk
862   17         120           80   6.7     102.0         76   mid risk

[4965 rows x 7 columns]


(4965, 7)

In [ ]:
data1 = fetal_data[fetal_data['fetal_health'] == 1.0]
data2 = fetal_data[fetal_data['fetal_health'] == 2.0]
data3 = fetal_data[fetal_data['fetal_health'] == 3.0]
print(data1.shape)
print(data2.shape)
print(data3.shape)

(1655, 22)
(295, 22)
(176, 22)


In [ ]:
data2_upsampled = resample(data2, replace=True, n_samples=len(data1), random_state=42)
data3_upsampled = resample(data3, replace=True, n_samples=len(data1), random_state=42)


In [ ]:
fetal_data_upsampled= pd.concat([data1,data2_upsampled,data3_upsampled])

In [ ]:
fetal_data_upsampled['fetal_health'].value_counts()

1.0    1655
2.0    1655
3.0    1655
Name: fetal_health, dtype: int64

## Data Shuffling_Fetal

In [ ]:
fetal_data = fetal_data_upsampled.sample(frac=1)
fetal_data.shape

(4965, 22)

In [ ]:
maternal_X = maternal_data.drop(["RiskLevel"], axis=1)
maternal_Y = maternal_data["RiskLevel"]

In [ ]:
label_mapping = {"low risk": 0, "mid risk": 1, "high risk": 2}

In [ ]:
maternal_Y = maternal_Y.map(label_mapping)

In [ ]:
print(maternal_Y)

374    2
410    1
84     1
676    2
297    0
      ..
618    0
67     1
16     2
505    0
862    1
Name: RiskLevel, Length: 4965, dtype: int64


## Preprocessing and Splitting _ Maternal

Features and labels are separated. Data is scaled using StandardScaler, and split into training and testing sets (70/30).

In [ ]:
from sklearn.model_selection import train_test_split
maternal_X_train, maternal_X_test, maternal_Y_train, maternal_Y_test = train_test_split(maternal_X,maternal_Y, test_size=0.3, random_state=0)
print('Shape of X_train:', maternal_X_train.shape)
print('Shape of y_train:', maternal_Y_train.shape)
print("Shape of X_test:", maternal_X_test.shape)
print("Shape of y_test:", maternal_Y_test.shape)

Shape of X_train: (3475, 6)
Shape of y_train: (3475,)
Shape of X_test: (1490, 6)
Shape of y_test: (1490,)


In [ ]:
print(maternal_X_test)

     Age  SystolicBP  DiastolicBP     BS  BodyTemp  HeartRate
19    10          70           50   6.90      98.0         70
90    22         120           90   7.80      98.0         82
763   12          95           60   6.90      98.0         65
648   17          90           60   9.00     102.0         86
931   22         100           65   7.50      98.0         70
..   ...         ...          ...    ...       ...        ...
229   15         120           80   7.20      98.0         70
468   25         140          100   6.80      98.0         80
91    60         120           85  15.00      98.0         60
966   40         120           95  11.00      98.0         80
682   25         140          100   7.01      98.0         80

[1490 rows x 6 columns]


In [ ]:
scaler = StandardScaler()
maternal_X_train = scaler.fit_transform(maternal_X_train)
maternal_X_test=scaler.transform(maternal_X_test)

##Maternal Model
Neural Network to classify maternal risk into 3 classes. Uses:

ReLU activations

Dropout for regularization

Softmax output for multi-class prediction

In [ ]:
maternal_model = Sequential([
Dense(128,input_shape = [maternal_X_train.shape[1],], activation = 'relu'),
Dropout(rate = 0.3),
Dense(128,activation = 'relu'),
Dropout(rate = 0.3),
Dense(3,activation = 'softmax')
])

In [ ]:
maternal_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               896       
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 3)                 387       
                                                                 
Total params: 17795 (69.51 KB)
Trainable params: 17795 (69.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
maternal_model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["Accuracy"])

In [ ]:
maternal_model.fit(maternal_X_train, maternal_Y_train, batch_size=32, epochs=100, validation_data=(maternal_X_test, maternal_Y_test),verbose=1)

Epoch 1/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2407 - Accuracy: 0.8944 - val_loss: 0.2227 - val_Accuracy: 0.8879
Epoch 2/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2371 - Accuracy: 0.8892 - val_loss: 0.2222 - val_Accuracy: 0.8946
Epoch 3/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2391 - Accuracy: 0.8889 - val_loss: 0.2198 - val_Accuracy: 0.8933
Epoch 4/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2317 - Accuracy: 0.8927 - val_loss: 0.2152 - val_Accuracy: 0.8919
Epoch 5/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2322 - Accuracy: 0.8915 - val_loss: 0.2126 - val_Accuracy: 0.8973
Epoch 6/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2327 - Accuracy: 0.8895 - val_loss: 0.2150 - val_Accuracy: 0.8940
Epoch 7/100
109/109 [==============================] - 0s 4ms/step - loss: 0.2322 - Accuracy: 0.8901 - val_loss: 0.2178 - val_Accuracy: 0.8893

In [ ]:
maternal_Y_pred = maternal_model.predict(maternal_X_test)

47/47 [==============================] - 0s 2ms/step


In [ ]:
maternal_Y_pred.shape

(1490, 3)

In [ ]:
fetal_X = fetal_data.drop(["fetal_health"], axis=1)
fetal_Y = fetal_data["fetal_health"]

In [ ]:
label_mapping = {1.0:0,2.0: 1,3.0: 2}

In [ ]:
fetal_Y = fetal_Y.map(label_mapping)

## Preprocessing and Training _ Fetal
Similar preprocessing and model training done for fetal data. Output is 3-class classification of fetal health status.

In [ ]:
from sklearn.model_selection import train_test_split
fetal_X_train, fetal_X_test, fetal_Y_train, fetal_Y_test = train_test_split(fetal_X,fetal_Y, test_size=0.3, random_state=0)
print('Shape of X_train:', fetal_X_train.shape)
print('Shape of y_train:', fetal_Y_train.shape)
print("Shape of X_test:", fetal_X_test.shape)
print("Shape of y_test:", fetal_Y_test.shape)

Shape of X_train: (3475, 21)
Shape of y_train: (3475,)
Shape of X_test: (1490, 21)
Shape of y_test: (1490,)


In [ ]:
scaler1 = StandardScaler()
fetal_X_train = scaler1.fit_transform(fetal_X_train)
fetal_X_test = scaler1.transform(fetal_X_test)

## Fetal Model


In [ ]:
fetal_model = Sequential([
    Dense(32,kernel_initializer='uniform', activation='relu', input_shape=(fetal_X_train.shape[1],)),
    Dense(64,kernel_initializer='uniform', activation='relu'),
    Dense(3,kernel_initializer='uniform', activation='softmax')])

In [ ]:
fetal_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 32)                704       
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 3)                 195       
                                                                 
Total params: 3011 (11.76 KB)
Trainable params: 3011 (11.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
fetal_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=["Accuracy"])


In [ ]:
fetal_model.fit(fetal_X_train, fetal_Y_train, batch_size=20, epochs=100, validation_data=(fetal_X_test, fetal_Y_test),verbose=1)

Epoch 1/100
174/174 [==============================] - 1s 4ms/step - loss: 0.6869 - Accuracy: 0.7376 - val_loss: 0.3985 - val_Accuracy: 0.8456
Epoch 2/100
174/174 [==============================] - 1s 3ms/step - loss: 0.3607 - Accuracy: 0.8558 - val_loss: 0.3257 - val_Accuracy: 0.8644
Epoch 3/100
174/174 [==============================] - 1s 3ms/step - loss: 0.3061 - Accuracy: 0.8771 - val_loss: 0.2982 - val_Accuracy: 0.8799
Epoch 4/100
174/174 [==============================] - 1s 3ms/step - loss: 0.2769 - Accuracy: 0.8904 - val_loss: 0.2704 - val_Accuracy: 0.8973
Epoch 5/100
174/174 [==============================] - 0s 3ms/step - loss: 0.2548 - Accuracy: 0.8953 - val_loss: 0.2588 - val_Accuracy: 0.8899
Epoch 6/100
174/174 [==============================] - 1s 3ms/step - loss: 0.2354 - Accuracy: 0.9096 - val_loss: 0.2395 - val_Accuracy: 0.9047
Epoch 7/100
174/174 [==============================] - 1s 3ms/step - loss: 0.2124 - Accuracy: 0.9180 - val_loss: 0.2277 - val_Accuracy: 0.9121

In [ ]:
fetal_Y_pred = fetal_model.predict(fetal_X_test)

47/47 [==============================] - 0s 2ms/step


##Prediction and Weighted Combination
Combine the predictions of both models using weighted averaging. This gives a continuous risk score scaled between 0 and 1.

In [ ]:
weight_model1 = 0.7
weight_model2 = 0.3
overall_risk = np.empty((1490, 1))
for i in range(1490):
  overall_pred = ((weight_model1 * maternal_Y_pred[i]) + (weight_model2 * fetal_Y_pred[i]))/2
  overall_risk_score = 1 - (overall_pred[0]+(overall_pred[1])/2)
  overall_risk[i] = overall_risk_score




[[0.65000004]
 [0.82464098]
 [0.60660661]
 ...
 [0.79373904]
 [0.87238758]
 [1.        ]]


In [ ]:
print(overall_risk.shape)

(1490, 1)


In [ ]:
print(maternal_X_test)
print(fetal_X_test)


[[-1.51374396 -2.34414113 -1.92722405 -0.61177391 -0.49907179 -0.56878401]
 [-0.62329378  0.29423813  0.87637766 -0.36140026 -0.49907179  0.8947661 ]
 [-1.3653356  -1.0249515  -1.22632363 -0.61177391 -0.49907179 -1.17859655]
 ...
 [ 2.19646515  0.29423813  0.52592744  1.64158896 -0.49907179 -1.78840909]
 [ 0.7123815   0.29423813  1.22682787  0.52881717 -0.49907179  0.65084108]
 [-0.40068123  1.34958983  1.57727809 -0.58117268 -0.49907179  0.65084108]]
[[-0.09703449  0.78587109 -0.22136783 ... -1.2726915   3.5726646
  -1.83553676]
 [ 0.89969876 -0.53276823 -0.22136783 ...  0.55687609 -0.59783669
  -1.83553676]
 [-1.59213437  0.12655143 -0.23710606 ... -0.62696176 -0.55200701
  -0.34139522]
 ...
 [ 1.59741204  2.76383008 -0.09546199 ...  1.52547069 -0.50617733
  -0.34139522]
 [ 0.30165881 -0.53276823 -0.22136783 ...  0.50306527 -0.55200701
   1.15274633]
 [-0.19670782 -0.53276823 -0.22136783 ... -1.48793475  0.43333121
  -0.34139522]]


In [ ]:
maternal_fetal_X = np.concatenate((maternal_X_test,fetal_X_test), axis=1)
print(maternal_fetal_data.shape)

(1490, 27)


In [ ]:
maternal_fetal_Y=overall_risk

In [ ]:
from sklearn.model_selection import train_test_split
maternal_fetal_X_train, maternal_fetal_X_test= train_test_split(maternal_fetal_X ,test_size=0.3, random_state=0)
print('Shape of X_train:', maternal_fetal_X_train.shape)
# print('Shape of y_train:', maternal_Y_train.shape)
print("Shape of X_test:", maternal_fetal_X_test.shape)
# print("Shape of y_test:", maternal_Y_test.shape)

Shape of X_train: (1043, 27)
Shape of X_test: (447, 27)


In [ ]:
from sklearn.model_selection import train_test_split
maternal_fetal_Y_train, maternal_fetal_Y_test= train_test_split(maternal_fetal_Y ,test_size=0.3, random_state=0)
print('Shape of X_train:', maternal_fetal_Y_train.shape)
# print('Shape of y_train:', maternal_Y_train.shape)
print("Shape of X_test:", maternal_fetal_Y_test.shape)
# print("Shape of y_test:", maternal_Y_test.shape)

Shape of X_train: (1043, 1)
Shape of X_test: (447, 1)


##Final Stacked Model
Use combined input features from both models to train a final model that predicts the integrated risk score. This model is trained as a regression model with sigmoid output.

In [ ]:
maternal_fetal_model = Sequential([
    Dense(32,kernel_initializer='uniform', activation='relu', input_shape=(maternal_fetal_X_train.shape[1],)),
    Dense(64,kernel_initializer='uniform', activation='relu'),
    Dense(1,kernel_initializer='uniform', activation='sigmoid')])

In [ ]:
maternal_fetal_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae', 'mse'])

In [ ]:
maternal_fetal_model.fit(maternal_fetal_X_train, maternal_fetal_Y_train, batch_size=20, epochs=100, validation_data=(maternal_fetal_X_test, maternal_fetal_Y_test),verbose=1)

Epoch 1/100
53/53 [==============================] - 1s 7ms/step - loss: 0.0518 - mae: 0.1850 - mse: 0.0518 - val_loss: 0.0134 - val_mae: 0.0950 - val_mse: 0.0134
Epoch 2/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0128 - mae: 0.0908 - mse: 0.0128 - val_loss: 0.0101 - val_mae: 0.0811 - val_mse: 0.0101
Epoch 3/100
53/53 [==============================] - 0s 4ms/step - loss: 0.0105 - mae: 0.0820 - mse: 0.0105 - val_loss: 0.0092 - val_mae: 0.0773 - val_mse: 0.0092
Epoch 4/100
53/53 [==============================] - 0s 3ms/step - loss: 0.0097 - mae: 0.0796 - mse: 0.0097 - val_loss: 0.0089 - val_mae: 0.0758 - val_mse: 0.0089
Epoch 5/100
53/53 [==============================] - 0s 3ms/step - loss: 0.0092 - mae: 0.0774 - mse: 0.0092 - val_loss: 0.0088 - val_mae: 0.0751 - val_mse: 0.0088
Epoch 6/100
53/53 [==============================] - 0s 3ms/step - loss: 0.0089 - mae: 0.0763 - mse: 0.0089 - val_loss: 0.0086 - val_mae: 0.0739 - val_mse: 0.0086
Epoch 7/100
53/53 [===

In [ ]:
maternal_fetal_Y_pred = maternal_fetal_model.predict(maternal_fetal_X_test)

14/14 [==============================] - 0s 2ms/step


In [ ]:
print(maternal_fetal_Y_pred)

[[0.8743919 ]
 [0.6523831 ]
 [0.7721001 ]
 [0.6930785 ]
 [0.784371  ]
 [0.6869591 ]
 [0.7855297 ]
 [0.8861874 ]
 [0.58969843]
 [0.96369064]
 [0.89086443]
 [0.6655473 ]
 [0.76731515]
 [0.7067877 ]
 [0.8726127 ]
 [0.5325791 ]
 [0.7949472 ]
 [0.6653547 ]
 [0.8657306 ]
 [0.8926094 ]
 [0.9729534 ]
 [0.72596854]
 [0.94901365]
 [0.97038263]
 [0.5804515 ]
 [0.9033298 ]
 [0.70717233]
 [0.8858016 ]
 [0.8385831 ]
 [0.7113851 ]
 [0.73041004]
 [0.7420931 ]
 [0.79962415]
 [0.7019292 ]
 [0.6743801 ]
 [0.6960992 ]
 [0.71466583]
 [0.7413292 ]
 [0.6216413 ]
 [0.69183594]
 [0.9454038 ]
 [0.5676778 ]
 [0.95992523]
 [0.6068243 ]
 [0.94079745]
 [0.7875306 ]
 [0.59982395]
 [0.656854  ]
 [0.7193363 ]
 [0.8958995 ]
 [0.90436286]
 [0.96375424]
 [0.873712  ]
 [0.68841493]
 [0.815857  ]
 [0.74285334]
 [0.73026854]
 [0.690982  ]
 [0.7309722 ]
 [0.7735265 ]
 [0.62724876]
 [0.9278874 ]
 [0.67963713]
 [0.77074045]
 [0.9628851 ]
 [0.71821594]
 [0.68219066]
 [0.6753199 ]
 [0.85721713]
 [0.5912059 ]
 [0.7591182 ]
 [0.96

##Prediction Function
A custom function that takes user input (both maternal and fetal), preprocesses it, and predicts the final risk score using the stacked model.

In [ ]:
def stacking(fetal,maternal):
  maternal= scaler.transform(maternal)
  fetal=scaler1.transform(fetal)
  maternal_fetal_data=np.concatenate((maternal,fetal), axis=1)
  maternal_fetal_result=maternal_fetal_model.predict(maternal_fetal_data)
  print(maternal_fetal_result)
fetal=[[120,0,0,0,0,0,0,73,0.5,43,2.4,64,62,126,2,0,120,137,121,73,1]]
maternal=[[90,10,20,10,10,7]]
stacking(fetal,maternal)




1/1 [==============================] - 0s 38ms/step
[[0.9440265]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
